# <font color = 'red'> Pré Processamento e Engenharia de Atributos</font>

### <font color = 'red'> Foco: </font>

- Pré processar o texto para a etapa de análise de sentimentos

- Remoção de stopwords e outros, para verificar ocorrência

In [1]:
# Imports
import os
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize

import spacy



import warnings
warnings.filterwarnings('ignore')


In [14]:
# Visualizando Cj dados
dados = pd.read_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/data_scraping_bruto_2hr.csv',
                       sep =',')

In [15]:
dados.head()

,Empresa,Relato,Avaliacao
0,Serasa Experian,"Boa tarde! Hoje, dia 02 de Dezembro de 2024, f...",Nota 5
1,Sky Airline,Bom dia. Eu e minha esposa ja fizemos algumas ...,Nota 1
2,Serasa Experian,Hoje 02/12/2024 entrei no Serasa e constava dí...,Nota 2
3,Equatorial Maranhão (CEMAR),"Minha energia foi cortada, solicito que seja r...",Nota 5
4,Serasa Experian,"CONFORME A DESCISAO DO STJ, DIVIDAS PRESCRITAS...",Nota 4


In [16]:
instituicoes_financeiras = ['Nubank',
              'Banco do Brasil',
              'Caixa Econômica Federal',
              'Banco Santander',
              'Banco Itaú Unibanco',
              'Mercado Pago',
              'Banco Inter (Banco Intermedium)',
              'Banco Bradesco',
              'C6 Bank',
              'PicPay',
              'Banco Pan']

In [17]:
# Selecionando apenas instituições financeiras

dados_financeiros = dados[dados['Empresa'].isin(instituicoes_financeiras)]

In [18]:
#Verificando ocorrências

dados_financeiros.Empresa.value_counts().reset_index(name='Quantidade Relatos')

,Empresa,Quantidade Relatos
0,Nubank,641
1,Banco do Brasil,541
2,Caixa Econômica Federal,465
3,Banco Santander,310
4,Mercado Pago,297
5,Banco Bradesco,278
6,Banco Itaú Unibanco,268
7,Banco Pan,235
8,Banco Inter (Banco Intermedium),217
9,C6 Bank,150


In [19]:
dados_financeiros.head(5)

,Empresa,Relato,Avaliacao
46,Nubank,Estão adicionando valores de juros abusivos e ...,Nota 1
47,Caixa Econômica Federal,"Eu fiz um empréstimo e atrasei para pagar, man...",Nota 4
117,Caixa Econômica Federal,"A loja da shoppe descontou 429,57 no meu Pix n...",Nota 5
124,Nubank,".Entendo que pelo pagamento ter sido recente, ...",Nota 5
133,Nubank,Foi solicitado a Nubank que retirassem mas os ...,Nota 1


In [20]:
dados_financeiros['Avaliacao'] = dados_financeiros['Avaliacao'].str.extract(r'(\d+)')

In [21]:
dados_financeiros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3531 entries, 46 to 21558
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Empresa    3531 non-null   object
 1   Relato     3531 non-null   object
 2   Avaliacao  3531 non-null   object
dtypes: object(3)
memory usage: 110.3+ KB


### <font color='green'> Remoção de pontuações e deixando o texto em caixa baixa

In [22]:
def processa_sentimento(text):
    # Minuscula
    text = text.lower()

    # Retirando pontuações, exceto "."
    text = re.sub(r'[^\w\s.]', '', text)

    # Retirando números
    text = re.sub(r'\d+', '', text)

    return text

In [23]:
dados_financeiros['Relato'] = dados_financeiros['Relato'].apply(processa_sentimento)

In [24]:
dados_financeiros.sample(15)

,Empresa,Relato,Avaliacao
11540,Caixa Econômica Federal,em . quitei o financiamento junto a caixa mas ...,5
9969,Banco Bradesco,no dia compareci acompanhado de minha mãe à a...,1
3834,Mercado Pago,no dia após finalizado dias do investimento ...,3
11144,Banco do Brasil,ha alguns meses tive problemas de saude na fam...,1
9012,Nubank,recebi um email da nubank no dia as para rea...,1
17196,Nubank,nu bank transformou minha dívida de r . em mai...,1
15885,Banco do Brasil,prezados conforme resolução nº . de de março ...,1
15532,Banco Inter (Banco Intermedium),meu nome consta no registrato do banco central...,5
518,C6 Bank,sou cliente c a mais de anos nunca atrasei d...,1
11141,C6 Bank,fui vitima de estelionato no dia entrei em co...,1


In [25]:
dados_financeiros.to_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_financeiros_sentimentos.csv',
                       sep =',', index = False, encoding = 'utf-8')


### <font color='green'> Pré processamento para frequência - TF IDF e Collocations

- Tokennizando os textos

In [26]:
word_tokenize("Data Science Academy 2024")

['Data', 'Science', 'Academy', '2024']

In [27]:
# Carregar modelo em português
lematizacao = spacy.load("pt_core_news_sm") 

# Aplicar lematização em cada relato
dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lambda tokens: [token.lemma_ for token in lematizacao(" ".join(tokens))])

In [29]:
nlp = spacy.load("pt_core_news_sm")

# Função para lematizar uma frase completa
def lematizar_frase(frase):
    doc = nlp(frase)
    return " ".join([token.lemma_ for token in doc])

dados_financeiros['Relato_Lematizado'] = dados_financeiros['Relato'].apply(lematizar_frase)

# Exibir o DataFrame resultante
print(dados_financeiros[['Empresa', 'Relato', 'Avaliacao', 'Relato_Lematizado']])


                       Empresa  \
46                      Nubank   
47     Caixa Econômica Federal   
117    Caixa Econômica Federal   
124                     Nubank   
133                     Nubank   
...                        ...   
21534                  C6 Bank   
21539          Banco Santander   
21545          Banco do Brasil   
21547                  C6 Bank   
21558                  C6 Bank   

                                                  Relato Avaliacao  \
46     estão adicionando valores de juros abusivos e ...         1   
47     eu fiz um empréstimo e atrasei para pagar mand...         4   
117    a loja da shoppe descontou  no meu pix não com...         5   
124    .entendo que pelo pagamento ter sido recente p...         5   
133    foi solicitado a nubank que retirassem mas os ...         1   
...                                                  ...       ...   
21534  o banco reduziu meu limite sem nenhum comunica...         1   
21539  efetuei o pagamento do a

In [99]:
dados_sem_sw = dados_financeiros.copy()

In [100]:
len(dados_financeiros['Relato'].iloc[0])

195

In [101]:
len(dados_financeiros['Relato_Lematizado'].iloc[0])

199

In [102]:
dados_financeiros.head()

,Empresa,Relato,Avaliacao,Relato_Lematizado
23,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3,não consigir crédito pra fazer um empréstimo m...
41,Nubank,ja houve o pagamento da negociacao,2,ja haver o pagamento de o negociacao
45,Nubank,exclusao do nome retirar negativacao do nome s...,1,exclusao de o nome retirar negativacao de o no...
49,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1,ser cliente de o c6 bank haver tempo sempre ma...
52,Nubank,gostaria de reclamar sobre essa dívida quer es...,5,gostar de reclamar sobre esse dívida querer es...


- Tratamento de StopWords

In [ ]:
from nltk.tokenize import word_tokenize


def tokenizando(texto):
 array_tokenizado = word_tokenize(texto)
 return array_tokenizado

In [114]:
dados_sem_sw['Relato_Lematizao_SW'] = dados_sem_sw['Relato_Lematizado'].map(lambda x: tokenizando(x))

In [115]:
# Stopwords em portugues
stopwords_PT = set(stopwords.words("portuguese"))

In [116]:
# Função para verificar se está com Stopwords no relato dos clientes - Utilizando list comprehension

def veriricaStopWord(texto):
 texto_filtrado = [palavra for palavra in texto if palavra not in stopwords_PT]
 return texto_filtrado

In [117]:
dados_sem_sw

,Empresa,Relato,Avaliacao,Relato_Lematizado,Relato_Lematizao_SW
23,Caixa Econômica Federal,não consigo crédito pra fazer um empréstimo me...,3,não consigir crédito pra fazer um empréstimo m...,"[não, consigir, crédito, pra, fazer, um, empré..."
41,Nubank,ja houve o pagamento da negociacao,2,ja haver o pagamento de o negociacao,"[ja, haver, o, pagamento, de, o, negociacao]"
45,Nubank,exclusao do nome retirar negativacao do nome s...,1,exclusao de o nome retirar negativacao de o no...,"[exclusao, de, o, nome, retirar, negativacao, ..."
49,C6 Bank,sou cliente do c6 bank há tempos sempre manten...,1,ser cliente de o c6 bank haver tempo sempre ma...,"[ser, cliente, de, o, c6, bank, haver, tempo, ..."
52,Nubank,gostaria de reclamar sobre essa dívida quer es...,5,gostar de reclamar sobre esse dívida querer es...,"[gostar, de, reclamar, sobre, esse, dívida, qu..."
...,...,...,...,...,...
10910,Banco Itaú Unibanco,solicitando a retirada da restrição de apontam...,4,solicitar o retirada de o restrição de apontam...,"[solicitar, o, retirada, de, o, restrição, de,..."
10912,Banco Itaú Unibanco,estou com dificuldade para redefinir a senha d...,1,estar com dificuldade para redefinir o senha d...,"[estar, com, dificuldade, para, redefinir, o, ..."
10923,C6 Bank,gostaria que tirassem o apontamento no registr...,5,gostar que tirar o apontamento em o registrato...,"[gostar, que, tirar, o, apontamento, em, o, re..."
10924,Banco Bradesco,sou cliente do banco bradesco e possuo um cheq...,1,ser cliente de o banco bradesco e possuor um c...,"[ser, cliente, de, o, banco, bradesco, e, poss..."


In [149]:
dados_sem_sw['Relato_Lematizao_SW'] = dados_sem_sw['Relato_Lematizao_SW'].map(lambda x: veriricaStopWord(x))

In [150]:
dados_prontos = dados_sem_sw.copy()

In [151]:
# Remove numeros
def remove_numbers(tokens):
    return [word for word in tokens if not re.search(r'\d', word)]


# Aplicando a função no seu DataFrame
dados_prontos['Relato_Lematizao_SW'] = dados_prontos['Relato_Lematizao_SW'].apply(remove_numbers)

In [153]:
dados_prontos.sample(5)

,Empresa,Relato,Avaliacao,Relato_Lematizado,Relato_Lematizao_SW
10182,Banco Santander,quitei uma divida com um desconto que o propri...,3,quitar um divido com um desconto que o proprio...,"[quitar, divido, desconto, proprio, banco, for..."
2155,Nubank,olá gostaria que fizessem uma simulação corret...,1,olá gostar que fizer um simulação correto de o...,"[gostar, fizer, simulação, correto, fatura, ve..."
3344,Banco Inter (Banco Intermedium),meu nome continua com problemas pra conseguir ...,2,meu nome continuar com problema pra conseguir ...,"[nome, continuar, problema, conseguir, cartão,..."
10821,Mercado Pago,em 160924 fizemos uma compra de luvas no site ...,5,em 160924 fizemos um compra de luva em o site ...,"[fizemos, compra, luva, site, napoli, descartá..."
9492,C6 Bank,após uma recaída que levou meu filho pedro hac...,1,após um recaída que levar meu filho Pedro hach...,"[recaída, levar, filho, Pedro, hachiya, rangel..."


In [ ]:
# Adicionando novas Stopwords
stopwords_PT.update({'fiz', 'ja', 'olá', 'ter', 'dar', 'aqui', 'ir', 'dia', 'pra', 'uso', 'após', 'consigir', 'pois', 'abrir', 'pq'})

In [154]:
# Retirando colunas desnecessárias ao problema de negócio
dados_prontos.drop(columns={"Relato", "Avaliacao", "Relato_Lematizado"}, inplace=True)

In [156]:
dados_prontos.sample()

,Empresa,Relato_Lematizao_SW
7947,Banco Itaú Unibanco,"[entrar, contato, banco, solicitar, revogação,..."


In [157]:
dados_prontos.to_csv('/Projetos Pessoais/DataScience/PLN_Text_Analysis/data/dados_Frequency.csv',
                       sep =',', index = False, encoding = 'utf-8')